HW6 Мельчук А.Б.

# Вебинар 6. Двухуровневые модели рекомендаций


Код для src, utils, metrics вы можете скачать из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [2]:
!pip install implicit

     |████████████████████████████████| 1.1MB 2.8MB/s 
  Created wheel for implicit: filename=implicit-0.4.2-cp36-cp36m-linux_x86_64.whl size=3420110 sha256=b572523968aa45f95ba099b34e4ec10847d6cc0827d1f87e85606e55786d47e7
  Stored in directory: /root/.cache/pip/wheels/1b/48/b1/1aebe3acc3afb5589e72d3e7c3ffc3f637dc4721c1a974dff7
Successfully built implicit


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [2]:
data = pd.read_csv('./data/retail_train.csv')
item_features = pd.read_csv('./data/product.csv')
user_features = pd.read_csv('./data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)




In [3]:
n_items_before = data['item_id'].nunique()

data = prefilter_items(data, item_features=item_features, take_n_popular=5000)

n_items_after = data['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 47344 to 5001


In [4]:
# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
7,2375,26984851516,1,1085983,1,2.99,364,-0.4,1642,1,0.0,0.0,2.99
11,1364,26984896261,1,842930,1,2.19,31742,0.0,1520,1,0.0,0.0,2.19


In [5]:
recommender = MainRecommender(data_train_lvl_1)

GPU training requires factor size to be a multiple of 32. Increasing factors from 20 to 32.


In [6]:
recommender.get_als_recommendations(2375, N=200)

[1085983,
 824931,
 9677457,
 5566717,
 5566307,
 1087547,
 956666,
 1069531,
 1014948,
 8090519,
 1121028,
 981601,
 832442,
 909165,
 1106894,
 7441191,
 905087,
 1022428,
 1136043,
 989411,
 1024998,
 907366,
 860501,
 968687,
 844462,
 1101805,
 1081470,
 1090931,
 971053,
 1139493,
 1040842,
 6533936,
 5981074,
 837908,
 9835619,
 1079484,
 887503,
 9527494,
 1015597,
 1043920,
 834117,
 8090919,
 884518,
 951227,
 982955,
 839162,
 990004,
 6533362,
 7442008,
 956399,
 1003649,
 5563934,
 1008400,
 841686,
 851188,
 995198,
 926544,
 1121367,
 969836,
 968359,
 8090612,
 7441419,
 5568489,
 1024051,
 9270796,
 1029313,
 1132943,
 1083446,
 904129,
 1037417,
 6442569,
 9469110,
 9526794,
 1105426,
 846285,
 5569374,
 7441102,
 6534078,
 845774,
 9297136,
 918046,
 6904775,
 957317,
 1081533,
 870614,
 8090653,
 851129,
 1063707,
 912280,
 6704207,
 931736,
 988697,
 6533880,
 5570383,
 957063,
 8090560,
 1041758,
 915545,
 885939,
 7166866,
 932949,
 887474,
 1097661,
 8276172,
 8

In [7]:
recommender.get_own_recommendations(2375, N=200)

[1069531,
 6464080,
 956666,
 9527494,
 1121028,
 9677457,
 1046919,
 1087547,
 981601,
 1015474,
 1072685,
 1085983,
 930666,
 5570048,
 827919,
 970747,
 5569327,
 869835,
 1000753,
 1864101,
 1029743,
 1106523,
 5569230,
 916122,
 1044078,
 1070820,
 878996,
 1126899,
 8090521,
 866211,
 965267,
 1127831,
 844179,
 1020581,
 8090537,
 5569471,
 899624,
 930118,
 854852,
 986912,
 933835,
 6034857,
 834117,
 5585510,
 1081177,
 1075368,
 5569845,
 897954,
 983584,
 874972,
 1112238,
 957736,
 985999,
 1037840,
 1023720,
 1101010,
 1012587,
 1018740,
 957951,
 918846,
 5569374,
 940947,
 5568378,
 908318,
 854405,
 8090532,
 1122358,
 1040807,
 857503,
 1087102,
 910032,
 951412,
 950575,
 913210,
 1042907,
 1070702,
 883003,
 999104,
 965766,
 999858,
 976199,
 902172,
 838186,
 849202,
 872137,
 993638,
 1000753,
 865456,
 909714,
 935317,
 971922,
 825994,
 1098927,
 1057350,
 850102,
 904129,
 839419,
 1026118,
 999270,
 1055503,
 1138443,
 861821,
 1013928,
 968072,
 1043751,
 94

In [8]:
recommender.get_similar_items_recommendation(2375, N=200)

[1132771,
 999999,
 1042907,
 5573375,
 1041758,
 7139529,
 990004,
 844179,
 9445671,
 869388,
 1085983,
 1092264,
 861279,
 995198,
 844462,
 5568378,
 5569845,
 9526410,
 7025203,
 1029743,
 1106523,
 5569230,
 916122,
 1044078,
 1070820,
 878996,
 1126899,
 8090521,
 866211,
 965267,
 1127831,
 844179,
 1020581,
 8090537,
 5569471,
 899624,
 930118,
 854852,
 986912,
 933835,
 6034857,
 834117,
 5585510,
 1081177,
 1075368,
 5569845,
 897954,
 983584,
 874972,
 1112238,
 957736,
 985999,
 1037840,
 1023720,
 1101010,
 1012587,
 1018740,
 957951,
 918846,
 5569374,
 940947,
 5568378,
 908318,
 854405,
 8090532,
 1122358,
 1040807,
 857503,
 1087102,
 910032,
 951412,
 950575,
 913210,
 1042907,
 1070702,
 883003,
 999104,
 965766,
 999858,
 976199,
 902172,
 838186,
 849202,
 872137,
 993638,
 1000753,
 865456,
 909714,
 935317,
 971922,
 825994,
 1098927,
 1057350,
 850102,
 904129,
 839419,
 1026118,
 999270,
 1055503,
 1138443,
 861821,
 1013928,
 968072,
 1043751,
 944534,
 8858

In [10]:
recommender.get_similar_users_recommendation(2375, N=200)

ValueError: ignored

### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 200 кандидатов (k=200)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [11]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[823990, 856942, 859191, 953561, 980263, 99065..."
1,2,"[999999, 927360, 1042616, 1117358, 8090521, 12..."


In [14]:
# your_code
train_users = data_train_lvl_1['user_id'].unique()
result_lvl_1 = result_lvl_1[result_lvl_1['user_id'].isin(train_users)]

result_lvl_1['candidates_own'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=200))
result_lvl_1['candidates_als'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=200))
result_lvl_1['candidates_sim_items'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_items_recommendation(x, N=200))
# result_lvl_1['candidates_sim_users'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_users_recommendation(x, N=200))
result_lvl_1

,user_id,actual,candidates_own,candidates_als,candidates_sim_items
0,1,"[823990, 856942, 859191, 953561, 980263, 99065...","[1056402, 855601, 9297615, 856942, 1087895, 10...","[856942, 5577022, 9297615, 1082269, 825123, 99...","[1036297, 855601, 10149640, 5577022, 835278, 9..."
1,2,"[999999, 927360, 1042616, 1117358, 8090521, 12...","[244960, 901606, 12425418, 5567582, 978318, 55...","[244960, 5567582, 1053924, 9416729, 6408076, 9...","[1082212, 5569230, 5569845, 1070702, 6408076, ..."
2,3,"[1121321, 854405, 1008714, 12132312, 12263692,...","[1121321, 6391240, 1135622, 1118235, 9220882, ...","[933248, 957839, 12132312, 6513810, 9220882, 9...","[999999, 825731, 999999, 829001, 1070702, 1004..."
3,4,"[883932, 891423, 910109, 1056778, 1075368, 999...","[6391541, 9677093, 1013868, 997089, 7466859, 9...","[6391541, 910109, 902172, 933246, 856215, 9399...","[1107173, 844179, 995151, 1118046, 883003, 844..."
4,5,"[891061, 935393, 970747, 992765, 9244977, 9999...","[6904776, 835986, 992765, 1002499, 1130951, 87...","[977798, 6904776, 1004596, 9655369, 1139786, 8...","[1063760, 878996, 12301109, 1043751, 1139786, ..."
...,...,...,...,...,...
2045,2493,"[926905, 999999, 1097458, 1118033, 861494, 104...","[913038, 1030093, 1118264, 1039578, 911675, 98...","[1030093, 866158, 1037840, 1108296, 1118264, 9...","[999779, 1044078, 8090537, 899624, 1086119, 99..."
2046,2496,"[827919, 999999, 892531, 902094, 933067, 95648...","[859984, 1095486, 1077231, 7168058, 1037337, 9...","[922561, 1077717, 1077231, 1095486, 865026, 10...","[897954, 1044078, 865026, 831628, 968072, 1077..."
2047,2497,"[823990, 941361, 1029743, 1046545, 1051323, 11...","[1004436, 6904419, 843343, 901602, 871514, 929...","[1004436, 870568, 901602, 9297135, 6904419, 87...","[5569471, 999999, 850102, 844179, 866211, 5569..."
2048,2499,"[1013928, 919766, 952163, 1012587, 1131344, 71...","[1115160, 901460, 7025275, 8118596, 883665, 67...","[883665, 7025275, 937573, 1085983, 1011457, 81...","[878996, 999714, 8090537, 999999, 1138474, 913..."


In [15]:
result_lvl_1.apply(lambda row: recall_at_k(row['candidates_own'], row['actual'], 200), axis=1).mean()

0.3120545943179568

In [16]:
result_lvl_1.apply(lambda row: recall_at_k(row['candidates_als'], row['actual'], 200), axis=1).mean()

0.19187953521476292

In [17]:
result_lvl_1.apply(lambda row: recall_at_k(row['candidates_sim_items'], row['actual'], 200), axis=1).mean()

0.27425021996929744

In [19]:
# result_lvl_1.apply(lambda row: recall_at_k(row['candidates_sim_users'], row['actual'], 200), axis=1).mean()

In [20]:
result_lvl_1['candidates_own_20'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=20))
result_lvl_1['candidates_own_50'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))
result_lvl_1['candidates_own_100'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=100))
result_lvl_1['candidates_own_500'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=500))

In [21]:
result_lvl_1.apply(lambda row: recall_at_k(row['candidates_own_20'], row['actual'], 20), axis=1).mean()

0.11021375243753251

In [22]:
result_lvl_1.apply(lambda row: recall_at_k(row['candidates_own_50'], row['actual'], 50), axis=1).mean()

0.18594928624504428

In [23]:
result_lvl_1.apply(lambda row: recall_at_k(row['candidates_own_100'], row['actual'], 100), axis=1).mean()

0.2500790302666932

In [24]:
result_lvl_1.apply(lambda row: recall_at_k(row['candidates_own_500'], row['actual'], 500), axis=1).mean()

0.41161468460291073

Рекоммендации собственных товаров и схожик с ними дают значительно лучший результат. Качество модели растёт вместе с К.

### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [ ]:
# your_code

In [25]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

# Пока только warm start
train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=200))

In [26]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['drop'] = 1  # фиктивная пересенная

users_lvl_2.head(4)

,user_id,item_id,drop
0,2223,878222,1
0,2223,891463,1
0,2223,920426,1
0,2223,6442572,1


In [27]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('drop', axis=1, inplace=True)

In [28]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(data_train_lvl_2, on=['user_id', 'item_id'], how='left')


targets_lvl_2.head(20)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
0,2223,878222,1.0,2065,MEAT-PCKGD,National,HEAT/SERVE,CHICKEN-FULLY COOKED,27.07 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.909460e+10,159.0,1.0,3.99,381.0,-2.00,202.0,23.0,0.0,0.0,3.99
1,2223,891463,1.0,93,MEAT-PCKGD,National,DINNER SAUSAGE,SMOKED/COOKED,16 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.884073e+10,146.0,2.0,5.00,381.0,-0.98,54.0,22.0,0.0,0.0,2.50
2,2223,920426,0.0,69,DRUG GM,Private,BATTERIES,ALKALINE BATTERIES,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2223,6442572,0.0,510,SEAFOOD-PCKGD,National,SEAFOOD - FROZEN,SEAFOOD-FRZ-RAW FILLETS,16 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2223,847529,0.0,111,DRUG GM,National,CIGARETTES,CIGARETTES,CTN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2223,843911,0.0,320,GROCERY,National,FRZN MEAT/MEAT DINNERS,FRZN SS PREMIUM ENTREES/DNRS/T,14 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2223,826090,0.0,1251,GROCERY,National,FRZN MEAT/MEAT DINNERS,FRZN SS PREMIUM ENTREES/DNRS/T,24 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2223,825170,0.0,794,GROCERY,National,COLD CEREAL,ADULT CEREAL,24 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2223,988275,0.0,1643,DRUG GM,National,MAGAZINE,ETHNIC-MAGAZINE,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2223,902362,0.0,1251,GROCERY,National,FRZN MEAT/MEAT DINNERS,FRZN SS PREMIUM ENTREES/DNRS/T,16 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
user_features

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16
...,...,...,...,...,...,...,...,...
796,35-44,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2494
797,45-54,A,75-99K,Homeowner,Unknown,3,1,2496
798,45-54,U,35-49K,Unknown,Single Male,1,None/Unknown,2497
799,25-34,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2498


In [37]:
test = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
test

,user_id,item_id
0,1,"[823990, 856942, 859191, 953561, 980263, 99065..."
1,2,"[999999, 927360, 1042616, 1117358, 8090521, 12..."
2,3,"[1121321, 854405, 1008714, 12132312, 12263692,..."
3,4,"[883932, 891423, 910109, 1056778, 1075368, 999..."
4,5,"[891061, 935393, 970747, 992765, 9244977, 9999..."
...,...,...
2045,2493,"[926905, 999999, 1097458, 1118033, 861494, 104..."
2046,2496,"[827919, 999999, 892531, 902094, 933067, 95648..."
2047,2497,"[823990, 941361, 1029743, 1046545, 1051323, 11..."
2048,2499,"[1013928, 919766, 952163, 1012587, 1131344, 71..."
